In [1]:

import os
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
# from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
# import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_path = 'train_set.csv'
test_path = 'test_set.csv'

print("Train Set Exists:", os.path.exists(train_path))
print("Test Set Exists:", os.path.exists(test_path))

# Load the datasets
train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

Train Set Exists: True
Test Set Exists: True


In [3]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [4]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [5]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [54]:
# scalar=MinMaxScaler()
# X = scalar.fit_transform(X)
# X_testdata = scalar.transform(X_testdata)




In [6]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [7]:
# import xgboost as xgb
# from sklearn.model_selection import GridSearchCV

# # Define the parameter grid
# param_grid = {
#     'learning_rate': [0.01, 0.03, 0.1],
#     'max_depth': [4, 6, 8],
#     'n_estimators': [100, 200, 376],
#     'gamma': [0, 0.1, 0.2],
#     'reg_lambda': [0.01, 0.1, 1],
#     'min_child_weight': [1, 3, 5],
#     'colsample_bytree': [0.5, 0.7, 1.0],
#     'colsample_bylevel': [0.5, 0.7, 1.0],
#     'scale_pos_weight': [1, 1.1, 1.2],
#     'subsample': [0.7, 0.8, 1.0]
# }

# # Initialize the classifier
# clf = xgb.XGBClassifier(
#     objective='binary:logistic',
#     eval_metric='logloss',
#     random_state=42
# )

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)

# # Fit the model
# grid_search.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters found: ", grid_search.best_params_)

Fitting 3 folds for each of 59049 candidates, totalling 177147 fits


MemoryError: Unable to allocate 7.51 MiB for an array with shape (8, 123060) and data type int64

In [64]:
clf =xgb.XGBClassifier(
    learning_rate= 0.0334925,
    max_depth=6,
    n_estimators=376,
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42,
    gamma=0.097, 
    reg_lambda=0.0271605,
    min_child_weight=4.166362,
    colsample_bytree=0.673,
    colsample_bylevel= 0.65,
    scale_pos_weight= 1.103739,
    subsample=0.7967162407706075
)

clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.65, colsample_bynode=None,
              colsample_bytree=0.673, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.097, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.0334925, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=4.166362, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=376,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [65]:
# Get feature importances
feature_importances = clf.feature_importances_

# Create a DataFrame for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
print(feature_importances_df)

# Select the top N important features (e.g., top 10)
top_features = feature_importances_df['Feature'].head(10).tolist()

   Feature  Importance
67     X69    0.042398
68     X70    0.036581
24     X26    0.036149
16     X18    0.031331
18     X20    0.022904
..     ...         ...
3       X5    0.009058
53     X55    0.008582
75     X77    0.007680
69     X71    0.000000
74     X76    0.000000

[77 rows x 2 columns]


In [66]:
X_train_top = X_train[top_features]
X_test_top = X_test[top_features]
X_testdata_top = X_testdata[top_features]

# Re-train the model with the top features
clf.fit(X_train_top, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.65, colsample_bynode=None,
              colsample_bytree=0.673, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.097, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.0334925, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=4.166362, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=376,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [67]:
y_pred_proba = clf.predict_proba(X_test_top)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc}")

AUC: 0.9555739350894756


In [68]:
# Predict probabilities for the test set
y_pred_proba_testdata = clf.predict_proba(X_testdata_top)[:, 1]

# Create a DataFrame with RecordId and predicted probabilities
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

Probabilities saved to test_set_with_probabilities.csv
[1.1650516e-03 9.7855900e-03 5.0518580e-05 ... 2.3671605e-04 1.4366944e-04
 2.0424872e-04]
        RecordId  Y_probability
0         300001       0.001165
1         300002       0.009786
2         300003       0.000051
3         300004       0.000404
4         300005       0.000149
...          ...            ...
105477    405478       0.000025
105478    405479       0.142999
105479    405480       0.000237
105480    405481       0.000144
105481    405482       0.000204

[105482 rows x 2 columns]
